### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
    # ==0.7.3

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.7.3
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

# **Model**

Load up `DeepSeek-R1-Distill-Qwen1.5B`, and set parameters

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 4096 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,# Reduce if out of memory
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/deepseek-r1-distill-qwen-1.5b-unsloth-bnb-4bit
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Unsloth: vLLM loading unsloth/deepseek-r1-distill-qwen-1.5b-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.53 GB. Also swap space = 5 GB.
WARNING 04-25 11:19:35 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-25 11:19:46 config.py:549] This model supports multiple tasks: {'classify', 'generate', 'embed', 'score', 'reward'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection'], 'llm_int8_threshold': 

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:35<00:00,  1.33s/it]

INFO 04-25 11:20:47 model_runner.py:1562] Graph capturing finished in 36 secs, took 0.47 GiB
INFO 04-25 11:20:47 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 47.31 seconds


tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

## Evaluation

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
aime24 = load_dataset("simplescaling/aime24_nofigures")['train']
aime24

README.md:   0%|          | 0.00/833 [00:00<?, ?B/s]

aime24_nofigures.jsonl:   0%|          | 0.00/146k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'problem', 'solution', 'answer', 'url'],
    num_rows: 30
})

In [ ]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.0,
    max_tokens = max_seq_length,
)

In [ ]:
example = aime24[7]
prompt = tokenizer.apply_chat_template([
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)
print(prompt)
output = model.fast_generate(
    [prompt],
    sampling_params = sampling_params,
)[0].outputs[0].text
print(output)

<｜begin▁of▁sentence｜><｜User｜>There exist real numbers $x$ and $y$, both greater than 1, such that $\log_x\left(y^x\right)=\log_y\left(x^{4y}\right)=10$. Find $xy$.<｜Assistant｜><think>



Processed prompts: 100%|██████████| 1/1 [01:05<00:00, 65.40s/it, est. speed input: 0.84 toks/s, output: 61.80 toks/s]

Okay, so I have this problem here: There exist real numbers x and y, both greater than 1, such that log base x of (y^x) equals log base y of (x^{4y}) equals 10. I need to find the product xy. Hmm, let me try to figure this out step by step.

First, let me write down what's given. The problem states:

log_x(y^x) = 10 and log_y(x^{4y}) = 10.

So, both of these logarithms equal 10. I need to find xy.

I remember that logarithms can be tricky, but maybe I can use the change of base formula or some logarithmic identities to simplify these expressions.

Let me recall that log_b(a) = (ln a)/(ln b). So, maybe I can express both logarithms in terms of natural logarithms and set them equal to 10.

Starting with the first equation: log_x(y^x) = 10.

Using the logarithm power rule, log_b(a^c) = c * log_b(a). So, log_x(y^x) = x * log_x(y) = 10.

Similarly, the second equation: log_y(x^{4y}) = 10.

Again, using the power rule, this is 4y * log_y(x) = 10.

So now, I have two equations:

1) x * log_x(

# Run Till this cell

In [ ]:
i = 0
pretrained_outputs = []
for example in aime24:
    print(f"processing {i}-th example")
    text = tokenizer.apply_chat_template([
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)

    output = model.fast_generate(
      [text],
      sampling_params = sampling_params,
      lora_request = None,
      )[0].outputs[0].text
    pretrained_outputs.append(output)
    i +=1

processing 0-th example


Processed prompts: 100%|██████████| 1/1 [00:16<00:00, 16.76s/it, est. speed input: 9.67 toks/s, output: 74.95 toks/s]


processing 1-th example


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.58s/it, est. speed input: 15.74 toks/s, output: 75.54 toks/s]


processing 2-th example


...

Processed prompts: 100%|██████████| 1/1 [00:14<00:00, 14.24s/it, est. speed input: 6.95 toks/s, output: 75.28 toks/s]


processing 28-th example


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.65s/it, est. speed input: 21.86 toks/s, output: 75.63 toks/s]


processing 29-th example


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.17s/it, est. speed input: 21.09 toks/s, output: 75.27 toks/s]


In [ ]:
pretrained_results = []
for x,y in zip(aime24, pretrained_outputs):
  x['prediction'] = y
  x['pred_ans'] = extract_answer(y)
  pretrained_results.append(x)

for t in pretrained_results:
  print(t['answer'], t['pred_ans'])

204 204
113 23
371 36
385 12
110 12
104 89
721 17
025 25
809 1620
116 116
104 17
294 1365
540 10\sqrt{3239}
197 103
480 88
073 1156
468 384
601 3
023 3
321 809
211 -9323
315 24
236 246
045 18
033 33
080 120
055 10.00000000000000000000000000000000000001
699 1005
127 1
902 3840


### Training Data Prep
<a name="Data"></a>

In [ ]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""


In [ ]:
i = 0
outputs = []
for example in aime24:
    print(f"processing {i}-th example")
    text = tokenizer.apply_chat_template([
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)

    output = model.fast_generate(
      [text],
      sampling_params = sampling_params,
      lora_request = None,
      )[0].outputs[0].text
    outputs.append(output)
    i +=1